In [39]:
import pandas as pd

periods = 1000 #  for testing purposes, all is 1000

# Trading function

In [40]:
last_training = 621.38

def trading(df, n_ahead):
    adjusted_n_ahead = n_ahead-1 # bcuz lists start at 0
    df_adj = df.iloc[adjusted_n_ahead::63]

    latest_decision = 'hold'
    bank_start = 621.38
    bank = bank_start
    stock_count = 0
    nr_buys = 0
    nr_sells = 0

    for row, values in df_adj.iterrows():
        latest_close = last_training if values['time'] == 1 else df['Close'].iloc[row-adjusted_n_ahead-63]
        forecast = df['forecast'].iloc[row]
        expected = forecast - latest_close
        
        decision = 'hold'

        if expected > 0 and stock_count == 0:
            decision = 'buy'
            latest_decision = 'buy'
            bank -= latest_close
            stock_count += 1
            nr_buys += 1
        
        if expected < 0 and stock_count > 0:
            decision = 'sell'
            latest_decision = 'sell'
            bank += latest_close
            stock_count -= 1
            nr_sells += 1
        
        # print(decision, forecast, latest_close, bank)
    
    last_close = df['Close'].iloc[-63]
    if latest_decision == 'buy': # reverts decision if latest decision is buy
        bank += last_close
        nr_sells += 1

    buy_hold = last_close - last_training
    trading = bank - bank_start
    nr_holds = periods - nr_buys - nr_sells

    print(f'trading: {round(trading, 2)}, buys: {nr_buys}, sells: {nr_sells}, holds: {nr_holds}, yield: {round(bank/bank_start*100 - 100, 2)}%')
    print(f'buy&hold: {round(buy_hold, 2)} (buy: {last_training}, sell: {last_close}), yield: {round((last_close-last_training)/last_training*100, 2)}%')

# ARMA-GARCH-trading

In [41]:
# reading data
ag_df = pd.read_csv('data/r_cross_val.csv', usecols=[0,1,2])
ag_df = ag_df[ag_df['time'] <= periods]

In [42]:
trading(ag_df, 1)

trading: 732.61, buys: 78, sells: 78, holds: 844, yield: 117.9%
buy&hold: 384.64 (buy: 621.38, sell: 1006.02), yield: 61.9%
1353.9899999999998


In [43]:
trading(ag_df,3)

trading: 727.35, buys: 62, sells: 62, holds: 876, yield: 117.05%
buy&hold: 384.64 (buy: 621.38, sell: 1006.02), yield: 61.9%
1348.729999999999


In [44]:
trading(ag_df,5)

trading: 699.91, buys: 54, sells: 54, holds: 892, yield: 112.64%
buy&hold: 384.64 (buy: 621.38, sell: 1006.02), yield: 61.9%
1321.2899999999995


In [45]:
trading(ag_df, 21)

trading: 573.33, buys: 5, sells: 5, holds: 990, yield: 92.27%
buy&hold: 384.64 (buy: 621.38, sell: 1006.02), yield: 61.9%
1194.71


In [46]:
trading(ag_df, 63)

trading: 368.5, buys: 2, sells: 2, holds: 996, yield: 59.3%
buy&hold: 384.64 (buy: 621.38, sell: 1006.02), yield: 61.9%
989.88


# LSTM-trading

In [47]:
lstm_df = pd.read_csv('data/lstm_results.csv', usecols=[0,1,2])
lstm_df = lstm_df[lstm_df['time'] <= periods]

In [48]:
trading(lstm_df, 1)

trading: 77.84, buys: 51, sells: 51, holds: 898, yield: 12.53%
buy&hold: 384.64 (buy: 621.38, sell: 1006.02), yield: 61.9%
699.2199999999996


In [49]:
trading(lstm_df, 3)

trading: -2.6, buys: 87, sells: 87, holds: 826, yield: -0.42%
buy&hold: 384.64 (buy: 621.38, sell: 1006.02), yield: 61.9%
618.7799999999995


In [50]:
trading(lstm_df, 5)

trading: 47.79, buys: 121, sells: 121, holds: 758, yield: 7.69%
buy&hold: 384.64 (buy: 621.38, sell: 1006.02), yield: 61.9%
669.1699999999998


In [51]:
trading(lstm_df, 21)

trading: 150.27, buys: 137, sells: 137, holds: 726, yield: 24.18%
buy&hold: 384.64 (buy: 621.38, sell: 1006.02), yield: 61.9%
771.65


In [52]:
trading(lstm_df, 63)

trading: 51.33, buys: 188, sells: 188, holds: 624, yield: 8.26%
buy&hold: 384.64 (buy: 621.38, sell: 1006.02), yield: 61.9%
672.7100000000002
